In [24]:
import dictclient
from dictclient import Connection, Database
from sys import argv
import re
import sh

In [39]:
dictOut = sh.dict('weld')

In [234]:
class DictWord: 
    def __init__(self, word):
        self.word = word
        self.rawString = self.getDefinitions(word)
        if self.rawString is not None: 
            self.etymString = self.parseDefinition(self.rawString)
        else: 
            self.etymString = None    
            self.parentLang = None
            self.parent = None
        if self.etymString is not None: 
                self.parseEtym(self.etymString)

    def __repr__(self): 
        varDict = vars(self)
        return "\n".join(["%s: %s" % (key, val) 
                          for key, val in varDict.items()
                          if key is not 'rawString'])
    
    def getDefinitions(self, word): 
        try: 
            defs = sh.dict(word)
            return defs
        except:
            print('Error looking up: ', word)
            pass
        

    def parseDefinition(self, dictOut): 
        defList = dictOut.split('\n\n')
        if len(defList) > 0: 
            for definition in defList: 
                lines = definition.split('\n')
                # Etyms are usually on the second line
                if len(lines) > 1: 
                    secondLine = lines[1].strip()
                    # And they usually start with a pronunciation
                    if secondLine.startswith('('): 
                        match = re.search('\[(.*?)\]', secondLine)
                        if match is not None: 
                            # Grab the first one we see. 
                            return match.group(1).strip()

    def parseEtym(self, etym): 
        if etym == None: 
            return None
        languages = ["med.L.", "Arab.", "F.", "ON.", "ME.", "Norw.", "OCelt", "Oir.", "mod.Ir."
                    "OF.", "mod.F.", "L.", "OE", "OHG", "AngloFr.", "ONF", "Sp.", "It.", "OS.", 
                    "\(M\)LG.", "\(M\)Du.", "MHG"]
        matches = [re.search(lang, etym) for lang in languages]
        # Sort by where they appear in the list, so that we can pick 
        # the leftmost language. 
        sortedMatches = sorted(matches, key=lambda x: x.span()[0] if x is not None else 99999)
        if len(sortedMatches) > 0 and sortedMatches[0] is not None: 
            # Search after the name of the language
            langEnd = sortedMatches[0].span()[1] 
            etym = etym[langEnd:]
            # Get the first word after the language name. 
            match = re.search('([A-Za-z].*?)[,.;\s]', etym)
            if match is not None: 
                parent = match.group(1)
            else: 
                parent = None
            self.parentLang = sortedMatches[0].group(0) 
            self.parent = parent
        else: 
            self.parentLang = None
            self.parent = None

In [235]:
for w in testWords: 
    dw = DictWord(w)
    print(dw)
    print('\n')

word: devout
etymString: ME. devot, devout, a. OF. devot, devote (12th c. in Littré), = Pr. devot, Sp. devoto, It. divoto, ad. L. dēvōt-us devoted, given up by vow, pa. pple. of dēvovēre to devote. The close OF. ō became the vowel ou (uː) in ME., whence the modern diphthong ou; but a form in ō, Sc. oi, was also in use: see devote a.
parentLang: ME.
parent: devot


word: wonning
etymString: None


word: belt
etymString: Common Teut.: OE. bẻlt, cogn. with OHG. balz (? masc.), prob.:—OTeut. *baltjo-z, ad. L. balteus girdle. ON. has balti (neut.), perh. ad. L. balteum, common in med.L.
parentLang: OE
parent: bẻlt


word: would
etymString: The subj. of will v. 1 used substantively.
parentLang: None
parent: None


word: aller
etymString: None


word: damascene
etymString: ad. L. Damascēn-us, Gr. Δαμασκηνός of Damascus. Cf. Ger. damascen.
parentLang: L.
parent: Damascēn-us


word: natheless
etymString: f. OE. ná na adv. 1 + þe (= þý) the + læs less adv. Cf. netheless and notheless.
parentLa

word: reverence
etymString: a. OF. reverence (mod.F. révérence), or ad. L. reverentia (cf. Sp. and Pg. reverencia, It. re-, riverenza, -zia): see reverent a. and -ence.
parentLang: F.
parent: reverence


word: cologne
etymString: In F. Cologne, Ger. Köln, MG. Cölen, Du. Keulen:—L. Colonia Agrippina, the ancient Roman name.
parentLang: F.
parent: Cologne


word: temple
etymString: a. OF. temple fem. (11th c. in Roland), = Prov. templa, It. tempia:—pop. L. type *tempula, *templa, app. for cl.L. tempora, pl. of tempus `temple of the head' (taken later as fem. sing.: cf. Bible). OF. temple (still in Dict. Acad. 1694-1740) is represented in mod.F. by tempe (already in Palsgr., 1530).
parentLang: F.
parent: temple


word: indite
etymString: a. OF. enditer, -ditier, -ditter:—L. type *indictāre, f. in- (in- 2) + dictāre to declare, dictate, compose in words, freq. of dīcĕre to say. The same word orig. as indict, but retaining the French form of the radical part.
parentLang: F.
parent: enditer


word: sent
etymString: pa. pple. of send v.
parentLang: None
parent: None


word: army
etymString: a. F. armée, cogn. with Sp., Pg., Pr. armada, It. armata, subst. use of pa. pple. of L. armāre, to arm, lit. `act of arming, armament, armed force.' The concrete sense is late in Fr. and Eng., and occurs first in reference to a naval force; cf. armada.
parentLang: F.
parent: armée


word: purchaser
etymString: ME., a. AF. purchasour, = OF. porchaceor, later pur-, pourchaseur, agent-n. from porchacier, pourchasser to purchase.
parentLang: ME.
parent: a


Error looking up:  alisandre
word: alisandre
etymString: None
parentLang: None
parent: None


word: curious
etymString: a. OF. curius (Ch. de Rol., 11th c.) = Pr. curios, Sp. and It. curioso:—L. cūriōs-us used only subjectively `full of care or pains, careful, assiduous, inquisitive'; French has also the objective sense in 14th c. (robes curieuses).    A word which has been used from time to time with many shades of meaning; the only sense

In [236]:
oed = open('oed.dict').read()

In [228]:
d = DictWord('wonning')

In [229]:
d

word: wonning
etymString: None